In [1]:
from pathlib import Path
import subprocess
import pandas as pd
import json, pickle,sys,glob
import simplekml, xmltodict
from shapely.geometry import Polygon, Point
import yaml
from mymodules.utils import *
from mymodules.myscripts import kml_mapper
from mymodules.aisight.naqsha import *

In [2]:
# base directory
BASE_DIR = Path().parent.resolve()

with open("config.yaml", "r") as yf:
    config = yaml.safe_load(yf)

# just to add collect script available on path
sys.path.append(Path(config["paths"]["mymodules"]).resolve())

In [3]:
# reading config from yaml
company = config["analysis"]["company"]
country = config["analysis"]["country"]
area_name = config["analysis"]["area"]

# output folder
# output_folder = Path(config["paths"]["data"]["output"]).resolve()

In [ ]:
# update paths section
config['paths'] = replace_placeholders(config['paths'], company, country)

print(yaml.dump(config, default_flow_style=False))

In [ ]:
geo_dir = Path(config["paths"]['data']).resolve() / area_name
geo_dir

In [ ]:
boundary, ptif, sec, whitespace = kml_mapper.setup(geo_dir)

In [ ]:
'''
get intersected box of tif with boundry
'''

In [ ]:
boundary._polygons[0]
TIF_data = ptif.to_dict(boundary._polygons[0])
TIF_data[0]

In [ ]:
boundary_polygon = boundary._polygons[0]
TIF_data = ptif.to_dict()
output_dict = []
for i in TIF_data:
    coordinates = TIF_data[i]['coordinates']
    population = TIF_data[i]['population']
    pixel_poly = Polygon(coordinates)
    area_percent = (
        pixel_poly.intersection(boundary_polygon).area / pixel_poly.area
    )

    name=round(population*area_percent)
    name_initial = int(name)
    name = (name_initial / 100) * 255
    name = hex(int(name)).split('x')[-1].zfill(2)
    
    if name_initial > 100: color = f'ff0000ff'
    else: color = f'ff0000{name}'
    output_dict.append({
        "coordinates": [
            {"lat": i[1], "lng": i[0]} for i in coordinates
        ],
        "population" : population,
        "color" : color
    })

In [2]:
df = pd.read_csv('./extra/Untitled 22_Makkah region_Xmap.csv') 
df.to_excel('./extra/22_Makkah region_Xmap.xlsx',index=False)